In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt


SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd() #get current working directory
#physical_devices = tf.config.list_physical_devices('GPU') 
physical_devices = tf.config.list_physical_devices('CPU') 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install git+https://github.com/rstrudel/segmenter.git
#!pip install .

In [ ]:
#!pip install --ignore-installed PyYAML 

     |████████████████████████████████| 596 kB 5.5 MB/s 


From https://blog.tensorflow.org/2018/07/an-introduction-to-biomedical-image-analysis-tensorflow-dltk.html

Using a TFRecords database: For most deep learning problems on image volumes, the database of training examples is too large to fit into memory. The TFRecords format allows to serialise training examples and store them on disk with quick write access (i.e. parallel data reads):

In [ ]:
#!mkdir -p '/content/drive/My Drive/Unzipped'
#%cd '/content/drive/My Drive/Unzipped' 
#!unzip '/content/drive/My Drive/Dataset/brats2019_slice_testing_tfrecords' 
#!unzip '/content/drive/My Drive/Dataset/brats2019_slice_training_tfrecords' 
#!unzip '/content/drive/My Drive/Dataset/brats2019_slice_validation_tfrecords' 

/content/drive/My Drive/Unzipped
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_testing_tfrecords.zip
replace brats2019_slice_testing_tfrecords/brats2019_slice_testing.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_training_tfrecords.zip
replace brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/My Drive/Dataset/brats2019_slice_validation_tfrecords.zip
replace brats2019_slice_validation_tfrecords/brats2019_slice_validation.tfrecords? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# RUNNING


In [ ]:
#if you want to clone the repo
#!git clone https://github.com/rstrudel/segmenter.git

In [ ]:
#%cd /content/drive/My Drive/Segmenter/segmenter



Col codice che segue vorremmo prendere i file di tipo tfrecords e decodificarli per ottenerne le immagini.
Le colormaps si possono trovare al sito https://scipy-cookbook.readthedocs.io/items/Matplotlib_Show_colormaps.html


In [ ]:
#!pip install tensorflow tensorflow_addons pillow numpy matplotlib

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np

#dataset_folder = os.path.join(cwd, "drive/My Drive/Unzipped/");
#dataset_name = os.path.join(dataset_folder, "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
dataset_name = os.path.join('/content/drive/My Drive/Unzipped/', "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
raw_dataset = tf.data.TFRecordDataset(dataset_name)
raw_dataset 
for raw_record in raw_dataset.take(10): #per capire la struttura del file tfrecord
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    #print(example)
    #print(repr(raw_record))


    parsed = tf.train.Example.FromString(raw_record.numpy())
    feature = parsed.features.feature

    raw_img_flair_mri = feature['flair_mri'].bytes_list.value[0]
    raw_img_seg = feature['seg'].bytes_list.value[0]
    raw_img_t1_mri = feature['t1_mri'].bytes_list.value[0]
    raw_img_t1ce_mri = feature['t1ce_mri'].bytes_list.value[0]
    raw_img_t2_mri = feature['t2_mri'].bytes_list.value[0]

    img_flair_mri = tf.image.decode_png(raw_img_flair_mri)
    img_t1_mri = tf.image.decode_png(raw_img_t1_mri)
    img_t1ce_mri = tf.image.decode_png(raw_img_t1ce_mri)
    img_t2_mri = tf.image.decode_png(raw_img_t2_mri)
    img_seg = tf.image.decode_png(raw_img_seg)

    f, splt = plt.subplots(1,5, figsize = (25.0, 25.0)) 

    
    splt[0].imshow(tf.squeeze(img_flair_mri), cmap='jet', vmin=0, vmax=255)
    splt[1].imshow(tf.squeeze(img_t1_mri), cmap='jet')
    splt[2].imshow(tf.squeeze(img_t2_mri), cmap='jet')
    splt[3].imshow(tf.squeeze(img_t1ce_mri), cmap='jet')
    splt[4].imshow(tf.squeeze(img_seg), cmap='jet')



Output hidden; open in https://colab.research.google.com to view.

A questo punto abbiamo le risonanze magnetiche in formato di immagini. Dobbiamo però ancora
- mettere queste immagini in dei tensori
- capire come dare in pasto queste immagini al segmenter 
  - NB: il segmenter accetta immagini a 3 canali, una per ogni colore, mentre noi qua abbiamo 4 immagini ad un canale ciascuna. Come gestiamo questo problema?
    - idea: una patch a 3 canali, prima di essere data in pasto all'encoder del transformer, viene comunque "appiattita" (flattened). Quindi, potremmo fare finta che le 4 "viste" della RM siano i canali, e invece di avere 3 canali ne abbiamo 4. A livello matematico non cambia nulla. Però: come modificare il segmenter per fare in modo che ciò sia possibile?
- ammesso che riusciamo ad utilizzare il segmenter per fare ciò: come facciamo poi a fare fine tuning? 

In [ ]:
#!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i images/ -o segmaps/

In [17]:
%cd /content/drive/My Drive/Segmenter/segmenter

#!export DATASET='/content/drive/My Drive/Immagine/'
!pip install .
!pip install --ignore-installed PyYAML 

/content/drive/My Drive/Segmenter/segmenter
Processing /content/drive/My Drive/Segmenter/segmenter
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 376 kB 12.7 MB/s 
     |████████████████████████████████| 312 kB 69.6 MB/s 
     |████████████████████████████████| 201 kB 68.8 MB/s 
     |████████████████████████████████| 190 kB 72.0 MB/s 
  Created wheel for segm: filename=segm-0.0.1-py3-none-any.whl size=1898 sha256=8f5b146ba42c51e3baec8bef502ff82d11dba5ac58c4ed206465551b800d6ab4
  Stored in directory: /tmp/pip-ephem-wheel-cache-iur77tmk/wheels/69/a6/f5/fedc521f36addf1abc849d46e38485fba3ad8d9a081418b659
  

Ora siamo pronti per effettuare una prova di segmentazione semantica. Se dovesse dire "AttributeError: module 'yaml' has no attribute 'FullLoader'", allora scommentare la prossima riga di codice ed eseguirla. È infatti un problema della versione di yaml.

In [ ]:
#set environment variables
os.environ['DATASET'] = "/content/drive/My Drive/Immagine"
#os.environ['FORCE_CUDA'] = "0"

Ora proviamo a fare una segmentazione di un'immagine normale

In [ ]:
%cd /content/drive/My Drive/Segmenter
#!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i images/ -o segmaps/ 
!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i "/content/drive/My Drive/Immagine"  -o segmaps/ 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.56s/it]


Col seguente codice proviamo a spacchettare le immagini dal dataset che mi ha dato il prof e creare un tensore a 4 canali da dare alla nostra versione "modificata" del segmenter, sotto la cartella my_segmenter. 
Tale versione modificata utilizzerà 4 canali invece che 3


In [13]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from PIL import Image
#dataset_folder = os.path.join(cwd, "drive/My Drive/Unzipped/");
#dataset_name = os.path.join(dataset_folder, "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
dataset_name = os.path.join('/content/drive/My Drive/Unzipped/', "brats2019_slice_training_tfrecords/brats2019_slice_training.tfrecords")
raw_dataset = tf.data.TFRecordDataset(dataset_name)

count = 0
for raw_record in raw_dataset: #per capire la struttura del file tfrecord
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())



    parsed = tf.train.Example.FromString(raw_record.numpy())
    feature = parsed.features.feature

    

    raw_img_flair_mri = feature['flair_mri'].bytes_list.value[0]
    raw_img_seg = feature['seg'].bytes_list.value[0]
    raw_img_t1_mri = feature['t1_mri'].bytes_list.value[0]
    raw_img_t1ce_mri = feature['t1ce_mri'].bytes_list.value[0]
    raw_img_t2_mri = feature['t2_mri'].bytes_list.value[0]

    
    img_flair_mri = tf.image.decode_png(raw_img_flair_mri)
    img_t1_mri = tf.image.decode_png(raw_img_t1_mri)
    img_t1ce_mri = tf.image.decode_png(raw_img_t1ce_mri)
    img_t2_mri = tf.image.decode_png(raw_img_t2_mri)
    img_seg = tf.image.decode_png(raw_img_seg)

    array = np.zeros([img_flair_mri.shape[0], img_flair_mri.shape[1], 4], dtype=np.uint8)

    array[:, :, 0] = tf.squeeze(img_flair_mri)
    array[:, :, 1] = tf.squeeze(img_t1_mri)
    array[:, :, 2] = tf.squeeze(img_t1ce_mri)
    array[:, :, 3] = tf.squeeze(img_t2_mri)

    img = Image.fromarray(array)
    img.save('/content/drive/My Drive/Test4Channels/test_4ch_{}.png'.format(count))
    count+=1
    



In [22]:
%cd /content/drive/My Drive/Segmenter/segmenter
#!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i "/content/drive/My Drive/Test4Channels"  -o segmaps/ 
!python -m segm.inference --model-path "/content/drive/My Drive/Models/Pre-trained/PASCAL - Seg-L-Mask16/checkpoint.pth" -i "/content/drive/My Drive/Dataset/ade20k/ADEChallengeData2016/images/training/"  -o segmaps/ 


/content/drive/My Drive/Segmenter/segmenter
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
  0%|                                      | 25/20210 [00:27<6:04:14,  1.08s/it]

Aborted!
